In [71]:
from clean import clean_mi_pl_bs, clean_mi_deferrals, clean_gross_tp_walk, clean_unearned_premium_unexpired_risk, clean_s2_summary_sheet, clean_version_control_solvency_balance_sheet, clean_version_control_scr_review, clean_version_control_s2_gaap, clean_version_control_s2_cap_prov_summary
import pandas as pd
import math
import numpy as np

In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
# pd.set_option('display.max_rows', None)
# pd.set_option('display.width', 1000)
# pd.set_option('display.max_colwidth', None)

In [4]:
all_s2_tables = clean_s2_summary_sheet(file_path = 'documents\\2506 MICL_S2 Gross TP walk.xlsx', sheet_name='S2 Balance Sheet Summary')

scr_summary = all_s2_tables['scr']

op_risk_calc = all_s2_tables['op_risk']

,OP risk calc,Final - As at 30th May 2025,Final - As at 30th June 2025,Movement
0,GEP - historic 1 to 12 months,79727936.81,76150932.45,-3.577004e+06
1,GEP - historic 13 - 24 months,131374797.43,125371386.41,-6.003411e+06
2,Gross TPs,198954923.028861,206279137.461737,7.324214e+06
4,Claim Op,5968647.690866,6188374.123852,2.197264e+05
5,Prem Op,54183.50112,55686.03624,1.502535e+03
6,30% of BSCR,5164158.738999,5187674.127955,2.351539e+04
7,OP risk,5164158.738999,5187674.127955,2.351539e+04
8,Op risk selection,BSCR,BSCR,1.000000e+00


dict_keys(['s2_balance_sheet', 'scr', 'nl_pr_risk', 'op_risk'])

In [6]:
final_scr = scr_summary.loc[scr_summary['SCR'] == 'SCR', 'Final - As at 30th June 2025'].item()
operational = scr_summary.loc[scr_summary['SCR'] == 'Operational', 'Final - As at 30th June 2025'].item()
capital_addon = scr_summary.loc[scr_summary['SCR'] == 'Capital add-on', 'Final - As at 30th June 2025'].item()
final_bscr = scr_summary.loc[scr_summary['SCR'] == 'Basic SCR', 'Final - As at 30th June 2025'].item()

if math.isclose(operational + capital_addon + final_bscr, final_scr):
    print('SCR Sum Checks Out')
else:
    print('SCR Sum Does Not Check Out')


SCR Sum Checks Out


Construct Operational

- **Inputs:**
    - **Gross Earned Premium (current year & prior year):** Sourced from `PL`.
    - **Gross Technical Provisions (non-life):** Sourced from `BS`.
    - **The BSCR:** The diversified sum of the Market, Counterparty, and Underwriting risk modules, calculated in the preceding step of the model.
    
- Calculation:
    1. **Calculate the Premium-Based Component (`Op_Prem`):** This step calculates a value based on the insurer's earned premiums and its recent growth.
        - **Formula:** `Op_Prem = 3% × GEP_CY + Max[0, 0.3% × (GEP_CY - 1.2 × GEP_PY)]`.
        - This means the model takes **3% of the gross earned premium from the last 12 months** and adds a small additional amount if premium volume has grown by more than 20% compared to the prior year, capturing the operational risk associated with rapid growth.
        
    2. **Calculate the Provisions-Based Component (`Op_Prov`):** This step calculates a value based on the size of the insurer's technical provisions.
        - **Formula:** `Op_Prov = 3% × Max(0, TP_Non-Life)`.
        - For a non-life insurer, this is **3% of the gross non-life technical provisions**.
        
    3. **Determine the Final SCR:** The model then combines these values with the BSCR.
        - **Formula:** `SCR_Operational = Min(0.3 × BSCR, Max(Op_Prem, Op_Prov))`.
        - The logic is to first take the **larger** of the premium-based and provisions-based components. This result is then compared to **30% of the BSCR**. The final Operational Risk SCR is the **smaller** of these two figures.

In [21]:
op_risk = op_risk_calc.loc[op_risk_calc['OP risk calc'] == 'OP risk', 'Final - As at 30th June 2025'].item()

gep_cy = op_risk_calc.loc[op_risk_calc['OP risk calc'] == 'GEP - historic 1 to 12 months', 'Final - As at 30th June 2025'].item() # gross earned premium in the past 12 months
gep_py = op_risk_calc.loc[op_risk_calc['OP risk calc'] == 'GEP - historic 13 - 24 months', 'Final - As at 30th June 2025'].item() # gross earned premiums in the 12 months preceding the past 12 months
op_prem = 0.03 * gep_cy + max(0.03 * (gep_cy - 1.2 * gep_py), 0)

tp_non_life = op_risk_calc.loc[op_risk_calc['OP risk calc'] == 'Gross TPs', 'Final - As at 30th June 2025'].item() # technical provisions recorded in the non-life underwriting module
tp_life = 0 # technical provisions recorded in the life underwriting module
op_prov = 0.03 * max(0, tp_non_life) + 0.045 * max(0, tp_life)

calc_operational = min(0.3 * final_bscr, max(op_prem, op_prov))

if math.isclose(calc_operational, op_risk):
    print('Operational Risk Checks Out')
else:
    print('Operational Risk Does Not Check Out')

Operational Risk Checks Out


Source `gep_cy`, `gep_py` and `tp_non_life` myself.

In [22]:
mi_pl = clean_mi_pl_bs(file_path = 'documents\\Jun25 MICL MI Pack v3.1 Isi pack exc PC.xlsx', sheet_name='PL')


Gross Written Premium 1 to 12 months

In [53]:
gross_written_premium = mi_pl.loc[mi_pl[('', 'Description')] == 'Gross written premium', ('YTD', 'Actual')]
gross_written_premium = gross_written_premium[:-1].item()
change_gross_provision_unearned_premiums = mi_pl.loc[mi_pl[('', 'Description')] == 'Change in the gross provision for unearned premiums', ('YTD', 'Actual')].item()
gross_earned_premium = gross_written_premium - change_gross_provision_unearned_premiums
print(gross_earned_premium)

24043974.87


Gross Written Premium 13 to 24 months

In [54]:
gross_written_premium_past = mi_pl.loc[mi_pl[('', 'Description')] == 'Gross written premium', ('', 'Prior_Year')]
gross_written_premium_past = gross_written_premium_past[:-1].item()
change_gross_provision_unearned_premiums_past = mi_pl.loc[mi_pl[('', 'Description')] == 'Change in the gross provision for unearned premiums', ('', 'Prior_Year')].item()
gross_earned_premium_past = gross_written_premium_past - change_gross_provision_unearned_premiums_past
print(gross_earned_premium_past)

73348932.74000001


Gross Technical Provisions

In [56]:
mi_bs = clean_mi_pl_bs(file_path = 'documents\\Jun25 MICL MI Pack v3.1 Isi pack exc PC.xlsx', sheet_name='BS')

c:\Users\georg\Desktop\mul_doc\repo\clean.py:18: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  top_header = top_header_raw.ffill().tolist()


In [59]:
provision_for_unearned_premiums = mi_bs.loc[mi_bs[('', 'Description')] == 'Provision for unearned premiums, gross amount', ('2025-06-01 00:00:00', 'Actual')].item()
claims_outstanding = mi_bs.loc[mi_bs[('', 'Description')] == 'Claims outstanding, gross amount', ('2025-06-01 00:00:00', 'Actual')].item()
ibnr = mi_bs.loc[mi_bs[('', 'Description')] == 'IBNR - Gross Amount', ('2025-06-01 00:00:00', 'Actual')].item()
gross_technical_provisions = provision_for_unearned_premiums + claims_outstanding + ibnr
print(gross_technical_provisions)

-261764341.87999997


BSCR

In [68]:
display(scr_summary)

,SCR,Item,Final - As at 30th April 2025,Final - As at 30th June 2025,Movement
0,Non-life,Premium & reserve risk,10365908.650862,10530272.232488,1.643636e+05
1,Non-life,Cat risk,559975.241914,559975.241914,0.000000e+00
2,Non-life,Lapse risk,0,0,0.000000e+00
3,Non-life,Diversification,-405999.837722,-406214.92895,-2.150912e+02
4,Non-life,Total,10519884.055055,10684032.545451,1.641485e+05
6,Counterparty,Type 1,3675997.468595,3637046.482763,-3.895099e+04
7,Counterparty,Type 2,359890.989,366670.218,6.779229e+03
8,Counterparty,Diversification,-82798.974687,-84156.890451,-1.357916e+03
9,Counterparty,Total,3953089.482909,3919559.810312,-3.352967e+04
11,Market,Interest rate,703768.605922,683872.125352,-1.989648e+04


In [87]:
labels = ['scr_market', 'scr_counterparty', 'scr_non_life'] # set order for risks; the correlation matrix is built with this order in mind

correlation_matrix_data = np.array([
    [1.00, 0.25, 0.25],
    [0.25, 1.00, 0.50],
    [0.25, 0.50, 1.00]
])

scr_values = {
    'scr_market': scr_summary.loc[(scr_summary['SCR'] == 'Market') & (scr_summary['Item'] == 'Total'), 'Final - As at 30th June 2025'].item(),
    'scr_counterparty': scr_summary.loc[(scr_summary['SCR'] == 'Counterparty') & (scr_summary['Item'] == 'Total'), 'Final - As at 30th June 2025'].item(),
    'scr_non_life': scr_summary.loc[(scr_summary['SCR'] == 'Non-life') & (scr_summary['Item'] == 'Total'), 'Final - As at 30th June 2025'].item()
}

scr_vector = np.array([scr_values[label] for label in labels]) # build scr_vector in order required to match correlation matrix

# BSCR = SQRT(Σi,j Corr(i,j) * SCRi * SCRj) can be expressed in matrix notation as: BSCR = SQRT(SCR_vector^T * Correlation_Matrix * SCR_vector) where ^T denotes the transpose of the vector.
term1 = scr_vector.T @ correlation_matrix_data # weigh each SCR value by its correlation to all other risks
variance = term1 @ scr_vector # dot product
calc_bscr = np.sqrt(variance) # brings back to original currency unit

bscr = scr_summary.loc[scr_summary['SCR'] == 'Basic SCR', 'Final - As at 30th June 2025'].item()

if math.isclose(calc_bscr, bscr):
    print('BSCR Checks Out')
else:
    print('BSCR Does Not Check Out')

diversification = scr_summary.loc[scr_summary['SCR'] == 'Diversification', 'Final - As at 30th June 2025'].item()

calc_diversification = calc_bscr - sum(scr_vector)

if math.isclose(calc_diversification, diversification):
    print('Diversification Checks Out')
else:
    print('Diversification Does Not Check Out')

BSCR Checks Out
Diversification Checks Out


Market

In [88]:
labels = ['interest_risk', 'equity_risk', 'spread_risk', 'concentration_risk', 'currency_risk'] # set order for risks; the correlation matrix is built with this order in mind

correlation_matrix_data = np.array([
    [1.00, 0.25, 0.25],
    [0.25, 1.00, 0.50],
    [0.25, 0.50, 1.00]
])

scr_values = {
    'scr_market': scr_summary.loc[(scr_summary['SCR'] == 'Market') & (scr_summary['Item'] == 'Total'), 'Final - As at 30th June 2025'].item(),
    'scr_counterparty': scr_summary.loc[(scr_summary['SCR'] == 'Counterparty') & (scr_summary['Item'] == 'Total'), 'Final - As at 30th June 2025'].item(),
    'scr_non_life': scr_summary.loc[(scr_summary['SCR'] == 'Non-life') & (scr_summary['Item'] == 'Total'), 'Final - As at 30th June 2025'].item()
}

scr_vector = np.array([scr_values[label] for label in labels]) # build scr_vector in order required to match correlation matrix

# BSCR = SQRT(Σi,j Corr(i,j) * SCRi * SCRj) can be expressed in matrix notation as: BSCR = SQRT(SCR_vector^T * Correlation_Matrix * SCR_vector) where ^T denotes the transpose of the vector.
term1 = scr_vector.T @ correlation_matrix_data # weigh each SCR value by its correlation to all other risks
variance = term1 @ scr_vector # dot product
calc_bscr = np.sqrt(variance) # brings back to original currency unit

bscr = scr_summary.loc[scr_summary['SCR'] == 'Basic SCR', 'Final - As at 30th June 2025'].item()

if math.isclose(calc_bscr, bscr):
    print('BSCR Checks Out')
else:
    print('BSCR Does Not Check Out')

diversification = scr_summary.loc[scr_summary['SCR'] == 'Diversification', 'Final - As at 30th June 2025'].item()

calc_diversification = calc_bscr - sum(scr_vector)

if math.isclose(calc_diversification, diversification):
    print('Diversification Checks Out')
else:
    print('Diversification Does Not Check Out')

np.float64(22825464.005657233)